In [ ]:
!pip3 install Flask

In [ ]:
!pip3 install sentencepiece

In [ ]:
!pip3 list

In [4]:
!huggingface-cli login 

^C


In [ ]:
!pip3 install -U transformers

In [ ]:
!pip3 install --upgrade pip

In [4]:
!pip3 cache purge

Files removed: 2401 (8220.6 MB)


In [ ]:
!pip3 install --upgrade transformers

In [ ]:
import os
import torch
import io
import scipy.io.wavfile
import numpy as np
from flask import Flask, request, jsonify, send_file
from transformers import (
    AutoProcessor,
    BarkModel,
    AutoModelForSpeechSeq2Seq,
    pipeline,
    Gemma3ForConditionalGeneration
)
from PIL import Image

# --- Flask Uygulamasını Başlatma ---
app = Flask(__name__)
# Geçici dosyaların yükleneceği klasörü oluştur
if not os.path.exists('uploads'):
    os.makedirs('uploads')

# --- Genel Ayarlar ve Cihaz Belirleme ---
# Modellerin CPU'da mı yoksa GPU'da mı çalışacağını belirle
device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
print(f"Kullanılan Cihaz: {device}")


# --- MODELLERİ YÜKLEME ---
# Modelleri uygulamanın başında YALNIZCA BİR KEZ yüklüyoruz.
# Bu, her istekte modeli yeniden yüklemenin önüne geçer ve performansı ciddi şekilde artırır.

print("Modeller yükleniyor... Bu işlem biraz zaman alabilir.")

# 1. Suno Bark Modelini Yükle (Text-to-Speech)
print("Suno Bark modeli yükleniyor...")
bark_model_id = "suno/bark"
bark_model = BarkModel.from_pretrained(bark_model_id).to(device)
bark_processor = AutoProcessor.from_pretrained(bark_model_id)
bark_sampling_rate = bark_model.generation_config.sample_rate
print("Suno Bark modeli yüklendi.")

# 2. OpenAI Whisper Modelini Yükle (Speech-to-Text)
print("OpenAI Whisper modeli yükleniyor...")
whisper_model_id = "openai/whisper-large-v3"
whisper_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    whisper_model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
).to(device)
whisper_processor = AutoProcessor.from_pretrained(whisper_model_id)
whisper_pipe = pipeline(
    "automatic-speech-recognition",
    model=whisper_model,
    tokenizer=whisper_processor.tokenizer,
    feature_extractor=whisper_processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)
print("OpenAI Whisper modeli yüklendi.")

# 3. Google Gemma Modelini Yükle (Text & Vision)
print("Google Gemma modeli yükleniyor...")
gemma_model_id = "model yolu"
gemma_model = Gemma3ForConditionalGeneration.from_pretrained(
    gemma_model_id, device_map="auto"
).eval()
gemma_processor = AutoProcessor.from_pretrained(gemma_model_id)
print("Google Gemma modeli yüklendi.")

print("\nTüm modeller başarıyla yüklendi. API kullanıma hazır.")


# --- API ENDPOINT'LERİ ---

@app.route('/')
def home():
    return "Yapay Zeka Modelleri API'sine hoş geldiniz! Kullanılabilir endpoint'ler: /gemma/generate-text, /gemma/generate-from-image, /whisper/transcribe, /bark/generate-speech"

# 1. Gemma ile Metinden Metin Üretme
@app.route('/gemma/generate-text', methods=['POST'])
def gemma_text_generation():
    data = request.get_json()
    if not data or 'prompt' not in data:
        return jsonify({"error": "Lütfen 'prompt' anahtarıyla bir metin gönderin."}), 400

    user_prompt = data['prompt']

    try:
        messages = [
            {"role": "user", "content": [{"type": "text", "text": user_prompt}]}
        ]

        inputs = gemma_processor.apply_chat_template(
            messages, add_generation_prompt=True, tokenize=True,
            return_dict=True, return_tensors="pt"
        ).to(gemma_model.device)

        input_len = inputs["input_ids"].shape[-1]

        with torch.inference_mode():
            generation = gemma_model.generate(**inputs, max_new_tokens=512, do_sample=True, temperature=0.7)
            generation = generation[0][input_len:]

        decoded_text = gemma_processor.decode(generation, skip_special_tokens=True)
        return jsonify({"response": decoded_text})

    except Exception as e:
        return jsonify({"error": f"Model işlenirken bir hata oluştu: {str(e)}"}), 500

# 2. Gemma ile Görsel ve Metinden Metin Üretme
@app.route('/gemma/generate-from-image', methods=['POST'])
def gemma_vision_generation():
    if 'image' not in request.files:
        return jsonify({"error": "Lütfen 'image' adıyla bir görsel dosyası yükleyin."}), 400

    image_file = request.files['image']
    prompt_text = request.form.get('prompt', 'Bu görseli detaylı bir şekilde açıkla.') # Varsayılan prompt

    try:
        image = Image.open(image_file.stream).convert("RGB")

        messages = [
            {
                "role": "user",
                "content": [
                    {"type": "image", "image": image},
                    {"type": "text", "text": prompt_text}
                ]
            }
        ]

        inputs = gemma_processor.apply_chat_template(
            messages, add_generation_prompt=True, tokenize=True,
            return_dict=True, return_tensors="pt"
        ).to(gemma_model.device)

        input_len = inputs["input_ids"].shape[-1]

        with torch.inference_mode():
            generation = gemma_model.generate(**inputs, max_new_tokens=512, do_sample=True, temperature=0.7)
            generation = generation[0][input_len:]

        decoded_text = gemma_processor.decode(generation, skip_special_tokens=True)
        return jsonify({"response": decoded_text})

    except Exception as e:
        return jsonify({"error": f"Model işlenirken bir hata oluştu: {str(e)}"}), 500

# 3. Whisper ile Sesten Metin Üretme
@app.route('/whisper/transcribe', methods=['POST'])
def whisper_transcription():
    if 'audio' not in request.files:
        return jsonify({"error": "Lütfen 'audio' adıyla bir ses dosyası yükleyin."}), 400

    audio_file = request.files['audio']
    # Dosyayı geçici olarak kaydet
    filepath = os.path.join("uploads", audio_file.filename)
    audio_file.save(filepath)

    try:
        # Whisper pipeline'ını çalıştır
        result = whisper_pipe(filepath, return_timestamps=True)
        transcription = result["text"]

        return jsonify({"transcription": transcription})
    except Exception as e:
        return jsonify({"error": f"Ses işlenirken bir hata oluştu: {str(e)}"}), 500
    finally:
        # Geçici dosyayı sil
        if os.path.exists(filepath):
            os.remove(filepath)

# 4. Suno Bark ile Metinden Ses Üretme
@app.route('/bark/generate-speech', methods=['POST'])
def bark_speech_generation():
    data = request.get_json()
    if not data or 'text' not in data:
        return jsonify({"error": "Lütfen 'text' anahtarıyla bir metin gönderin."}), 400

    text_prompt = data['text']
    # İsteğe bağlı olarak seslendirecek kişiyi de isteyebiliriz
    voice_preset = data.get('voice_preset', 'v2/tr_speaker_2')

    try:
        # Uzun metinleri modelin işleyebileceği parçalara böl
        max_chunk_length = 250
        text_chunks = [text_prompt[i:i+max_chunk_length] for i in range(0, len(text_prompt), max_chunk_length)]

        all_speech_outputs = []

        # ...
        for chunk in text_chunks:
            # 1. Adım: Önce tüm girdileri CPU'da oluştur.
            inputs = bark_processor(chunk, voice_preset=voice_preset, return_tensors="pt")
            
            # 2. Adım: Girdilerdeki her bir tensörü TEK TEK doğru cihaza taşı.
            # Bu, voice_preset'ten gelenlerin de taşınmasını garantiler.
            inputs_on_device = {key: value.to(device) for key, value in inputs.items()}

            # 3. Adım: Modeli, tümü aynı cihazda olan girdilerle çalıştır.
            speech_output = bark_model.generate(**inputs_on_device, do_sample=True)
            
            all_speech_outputs.append(speech_output[0].cpu().numpy())
        # ...

        # Tüm ses parçalarını birleştir
        combined_speech_output = np.concatenate(all_speech_outputs)

        # Sesi bir WAV dosyası olarak bellekte oluştur
        buffer = io.BytesIO()
        scipy.io.wavfile.write(buffer, rate=bark_sampling_rate, data=combined_speech_output)
        buffer.seek(0)

        # WAV dosyasını kullanıcıya gönder
        return send_file(
            buffer,
            mimetype='audio/wav',
            as_attachment=True,
            download_name='generated_speech.wav'
        )
    except Exception as e:
        return jsonify({"error": f"Ses üretilirken bir hata oluştu: {str(e)}"}), 500


# --- Uygulamayı Çalıştırma ---
if __name__ == '__main__':
    # host='0.0.0.0' ile ağdaki diğer cihazlardan da erişilebilir hale getirilir
    app.run(host='0.0.0.0', port=5000, debug=True, use_reloader=False)